# Задание 08

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx

from numpy import linalg as LA

## 1. Random walk normalized Laplacian
Нормализованный Лапласиан определяется:  

$$
L^{norm}=D^{-1}L= I-D^{-1}A, $$

$$
L^{norm}_{ij}=
 \begin{cases}
   1, &\text{  $i=j$, $k_i \neq 0$},\\
   -\displaystyle \frac{1}{k_i}, &\text{  $ i \neq j,  (i,j) \in E$},\\
   0, &\text{  otherwise}.\\
 \end{cases}
$$

* напишите функцию, вычисляющую матрицу $L_{norm}$ для сети;

In [3]:
def normalized_laplacian_matrix(G):
    return L

* убедитесь, что все собственные значения такой матрицы лежат $\left[0,2 \right]$.
* постройте спектры (огибающие гистограмм) сложных сетей, описывающие разные организмы (см. картинку 1 [статьи](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3888935/#B19)).

Если сети направленные или имеют мультиребра, предварительно сделайте их простыми (удалите направления и мультиребра). 

Похожи ли спектры различных организмов между собой? 

In [11]:
cat=nx.read_graphml('cat.graphml')
print(nx.number_of_nodes(cat),nx.number_of_edges(cat),nx.density(cat))

65 1139 0.27379807692307695


In [13]:
drs=nx.read_graphml('drosophila.graphml')
print(nx.number_of_nodes(drs),nx.number_of_edges(drs),nx.density(drs))

1781 33641 0.010611700282002914


## 2. Устойчивость коннектомов
Будем генерировать шум в сети двумы способами:

1) удалять ребро и ставить новое ребро между парой случайно выбранных вершин;

2) рандомизировать сеть, сохраняя степени вершин. 

* Постройте зависимости расстояния между исходным и зашумленным спектрами от числа шагов генерации шума. 

Расстояние определяется Евклидовой метрикой:

$$
s=\sum_{i}(\lambda_i-\lambda^{noise}_i)^2,
$$

Число шагов генерации шума измеряйте в количестве ребер графа: $N_{noise} = 0,~0.1E,~02E, ~ \dots E.$  

## 3. Схожесть спектров различных организмов

* Вычислите расстояние между спектрами коннектомов различных организмов;

Так как спектры имеют различную размерность, то Евклидова метрика не годится для измерения расстояния. Можно использовать [Earth mover distance](https://en.wikipedia.org/wiki/Earth_mover%27s_distance). 

Полезная функция (нужен scipy версии 1.0 и выше):

In [1]:
from scipy.stats import wasserstein_distance

* с чем можно сравнить найденные расстояния? 